In [274]:
#Importing required libraries 
import bamboolib as bam #easy to do data manipulation and highly recommend going to https://bamboolib.8080labs.com/ and that will help you follow along
import pandas as pd #data manipulation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm

In [275]:
patient_info = pd.read_csv(r'C:\Users\shyam\Downloads\XN Project\Data\patient_info(1).csv')

In [276]:
tbi_incident = pd.read_csv(r'C:\Users\shyam\Downloads\XN Project\Data\tbi_incident(1).csv')

In [277]:
new = pd.merge(tbi_incident, patient_info, how= 'inner',  on='patient_id')

In [278]:
new

      id tbi_incident_date                                     describe_event  \
0    212        1984-09-15  I was beat up by a gang age 11, then beat up b...   
1    366        1984-09-21  I was beat up by a gang age 11, and then beat ...   
2    236        2016-06-01  In my pool exercising after rehabilitation fro...   
3    202        2018-09-21  I was in a MVA. I was TBoned on my driver side...   
4    278        2020-02-16  My husband had an aneurysm  which led to a Mas...   
..   ...               ...                                                ...   
811  717        2020-06-20                             Motor vehicle accident   
812  388        2011-06-09  I was a passenger of a car that was hit by a d...   
813  495        1984-06-25  In a High altitude parachute accident in the Army   
814  802        2022-08-03                                 oyodrtdirktidrkyoi   
815  940        2016-02-18  I was standing in the doorway of my classroom ...   

             injury_from   

In [296]:
new.isna().sum()

id                              0
tbi_incident_date               0
injury_from                     0
head_hit_location               0
patient_id                      0
total_tbi                       0
immediate_symptoms_resulting    0
first_name                      0
last_name                       0
date_of_birth                   0
gender                          0
patient_type                    0
age                             0
cluster                         0
dtype: int64

In [280]:
new[new.duplicated(['id'], keep=False)]

Empty DataFrame
Columns: [id, tbi_incident_date, describe_event, injury_from, head_hit_location, patient_id, total_tbi, immediate_symptoms_resulting, first_name, last_name, date_of_birth, gender, patient_type, external_id]
Index: []

In [281]:
new= new.drop(columns =['describe_event', 'external_id'])

In [282]:
# Identify categorical columns with missing values
categorical_columns = ['head_hit_location', 'injury_from', 'patient_type', 'immediate_symptoms_resulting']  # Replace with your categorical column names

# Fill missing values with the mode
for column in categorical_columns:
    mode_value = new[column].mode()[0]
    new[column].fillna(mode_value, inplace=True)

In [283]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Convert 'date_of_birth' column to datetime
new['date_of_birth'] = pd.to_datetime(new['date_of_birth'])

# Calculate the age based on the current date
current_date = datetime.now()
new['age'] = new['date_of_birth'].apply(lambda x: relativedelta(current_date, x).years)



In [284]:
new.dtypes

id                                       int64
tbi_incident_date                       object
injury_from                             object
head_hit_location                       object
patient_id                              object
total_tbi                                int64
immediate_symptoms_resulting            object
first_name                              object
last_name                               object
date_of_birth                   datetime64[ns]
gender                                  object
patient_type                            object
age                                      int64
dtype: object

In [285]:
features = ['injury_from','head_hit_location','age', 'total_tbi']
target = 'gender'

In [286]:
X = new[features]  # Features
y = new[target]    # Target variable

# Perform one-hot encoding on the categorical features
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

# Print the encoded data
encoded_columns = encoder.get_feature_names_out(features)
encoded_data = pd.DataFrame(X_encoded.toarray(), columns=encoded_columns)
print(encoded_data)

     injury_from_& sports  injury_from_2nd was hemorrhagic stroke  \
0                     0.0                                     0.0   
1                     0.0                                     0.0   
2                     0.0                                     0.0   
3                     0.0                                     0.0   
4                     0.0                                     0.0   
..                    ...                                     ...   
811                   0.0                                     0.0   
812                   0.0                                     0.0   
813                   0.0                                     0.0   
814                   0.0                                     0.0   
815                   0.0                                     0.0   

     injury_from_Accident  injury_from_Accident, spoorts....  \
0                     0.0                                0.0   
1                     0.0                  

# KNN Model

In [287]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
X1_train, X1_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

knn1 = KNeighborsClassifier(n_neighbors=3)
knn1.fit(X1_train, y_train)
y_pred1 = knn1.predict(X1_test)
accuracy1 = accuracy_score(y_test, y_pred1)
print("Accuracy:", accuracy1)


Accuracy: 0.6979591836734694


In [288]:
from sklearn.metrics import classification_report
rf_y_pred = knn1.predict(X1_test)
rf_cm = confusion_matrix(y_test, rf_y_pred)
rf_cr = classification_report(y_test, rf_y_pred)

print("KNN Model Confusion Matrix:")
print(rf_cm)
print("KNN Model Classification Report:")
print(rf_cr)

KNN Model Confusion Matrix:
[[150  17   5]
 [ 45  19   1]
 [  4   2   2]]
KNN Model Classification Report:
              precision    recall  f1-score   support

      female       0.75      0.87      0.81       172
        male       0.50      0.29      0.37        65
       other       0.25      0.25      0.25         8

    accuracy                           0.70       245
   macro avg       0.50      0.47      0.48       245
weighted avg       0.67      0.70      0.67       245



# Random Forest Model

In [289]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7926829268292683


In [290]:
from sklearn.metrics import classification_report
rf_y_pred = rf_classifier.predict(X_test)
rf_cm = confusion_matrix(y_test, rf_y_pred)
rf_cr = classification_report(y_test, rf_y_pred)

print("Random Forest Confusion Matrix:")
print(rf_cm)
print("Random Forest Classification Report:")
print(rf_cr)

Random Forest Confusion Matrix:
[[114   5   0]
 [ 25  14   0]
 [  4   0   2]]
Random Forest Classification Report:
              precision    recall  f1-score   support

      female       0.80      0.96      0.87       119
        male       0.74      0.36      0.48        39
       other       1.00      0.33      0.50         6

    accuracy                           0.79       164
   macro avg       0.84      0.55      0.62       164
weighted avg       0.79      0.79      0.76       164



# SVM Model

In [291]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and train the Support Vector Machine Classifier
svm_classifier = SVC(kernel="rbf", gamma=0.5, C=1.0)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.75


In [292]:
from sklearn.metrics import classification_report
rf_y_pred = svm_classifier.predict(X_test)
rf_cm = confusion_matrix(y_test, rf_y_pred)
rf_cr = classification_report(y_test, rf_y_pred)

print("SVM Confusion Matrix:")
print(rf_cm)
print("SVM Classification Report:")
print(rf_cr)

SVM Confusion Matrix:
[[117   2   0]
 [ 33   6   0]
 [  6   0   0]]
SVM Classification Report:
              precision    recall  f1-score   support

      female       0.75      0.98      0.85       119
        male       0.75      0.15      0.26        39
       other       0.00      0.00      0.00         6

    accuracy                           0.75       164
   macro avg       0.50      0.38      0.37       164
weighted avg       0.72      0.75      0.68       164



In [318]:
new

      id tbi_incident_date          injury_from  \
0    212        1984-09-15              Assault   
1    366        1984-09-21              Assault   
2    236        2016-06-01  Subdural hemorrhage   
3    202        2018-09-21            Collision   
4    278        2020-02-16            Not known   
..   ...               ...                  ...   
811  717        2020-06-20             Accident   
812  388        2011-06-09             Accident   
813  495        1984-06-25             Accident   
814  802        2022-08-03             Military   
815  940        2016-02-18              Assault   

                                     head_hit_location  \
0    Front of Head,Left Side of Head,Top of Head,Ba...   
1    Assault, Back of Head, Confusion, Dazed or Vac...   
2                                         Back of Head   
3                                         Back of Head   
4                                         Back of Head   
..                                     

In [316]:
features = ['age']
target = 'gender'
X = new[features]  # Features
y = new[target]    # Target variable

# Perform one-hot encoding on the categorical features
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

In [324]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Split the data into training and testing sets (assuming X_encoded and y are already defined)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Perform K-means clustering
kmeans = KMeans(n_clusters=30)  # Specify the number of clusters
kmeans.fit(X_train)

# Get the predicted clusters for each data point
train_clusters = kmeans.predict(X_train)
test_clusters = kmeans.predict(X_test)

# Evaluate the quality of the clustering using silhouette score
train_silhouette = silhouette_score(X_train, train_clusters)
test_silhouette = silhouette_score(X_test, test_clusters)

print("Train Silhouette Score:", train_silhouette)
print("Test Silhouette Score:", test_silhouette)


Train Silhouette Score: 0.6868828513000281
Test Silhouette Score: 0.5920350609756101
